In [2]:
import os
import re
from pathlib import Path
import shutil

def get_markdown_files(root_dir):
    """Recursively find all markdown files excluding .gitbook directory"""
    for root, dirs, files in os.walk(root_dir):
        if '.gitbook' in dirs:
            dirs.remove('.gitbook')
        
        for file in files:
            if file.endswith('.md'):
                yield os.path.join(root, file)

def process_markdown_file(md_file):
    """Process a single markdown file and return required image replacements"""
    with open(md_file, 'r', encoding='utf-8') as f:
        content = f.read()
    
    base_name = os.path.splitext(os.path.basename(md_file))[0].replace('-', '_')
    
    # Updated pattern to match complete figure tags
    pattern = r'<figure><img src="(.*?\.png)".*?></figure>'
    matches = re.finditer(pattern, content)
    
    replacements = []
    for idx, match in enumerate(matches, 1):
        old_path = match.group(1)
        # Store markdown file for error reporting
        replacements.append((old_path, base_name, f"{base_name}_{idx}.png", md_file))
        
        # Determine if we need ../ or ../../ based on the original path
        dots = "../../" if old_path.startswith("../../") else "../"
        
        # Create complete replacement with proper GitBook structure
        new_tag = f'<figure><img src="{dots}.gitbook/assets/{base_name}_{idx}.png" alt=""><figcaption></figcaption></figure>'
        content = content.replace(match.group(0), new_tag)
    
    return content, replacements

# Start processing
try:
    current_dir = os.getcwd()
    image_replacements = []
    md_files_with_errors = set()
    
    # Process all markdown files
    for md_file in get_markdown_files(current_dir):
        print(f"Processing: {md_file}")
        new_content, replacements = process_markdown_file(md_file)
        
        with open(md_file, 'w', encoding='utf-8') as f:
            f.write(new_content)
        
        image_replacements.extend(replacements)
    
    assets_dir = os.path.join(current_dir, '.gitbook', 'assets')
    if os.path.exists(assets_dir):
        print("\nProcessing images in .gitbook/assets")
        temp_dir = os.path.join(assets_dir, 'temp')
        os.makedirs(temp_dir, exist_ok=True)
        
        # Process each image replacement
        for old_path, base_name, new_name, source_md in image_replacements:
            old_file = os.path.join(assets_dir, os.path.basename(old_path))
            if os.path.exists(old_file):
                new_file = os.path.join(temp_dir, new_name)
                shutil.copy2(old_file, new_file)
                print(f"Renamed: {os.path.basename(old_file)} -> {new_name}")
            else:
                rel_md_path = os.path.relpath(source_md, current_dir)
                md_files_with_errors.add((rel_md_path, old_path))
        
        # Remove old images
        for file in os.listdir(assets_dir):
            if file.endswith('.png'):
                try:
                    os.remove(os.path.join(assets_dir, file))
                except Exception as e:
                    print(f"Warning: Could not remove {file}: {str(e)}")
        
        # Move new images
        for file in os.listdir(temp_dir):
            try:
                shutil.move(os.path.join(temp_dir, file), os.path.join(assets_dir, file))
            except Exception as e:
                print(f"Warning: Could not move {file}: {str(e)}")
        
        # Clean up temp directory
        try:
            os.rmdir(temp_dir)
        except Exception as e:
            print(f"Warning: Could not remove temp directory: {str(e)}")
        
        # Print summary of errors
        if md_files_with_errors:
            print("\nMissing image references found in these files:")
            for rel_md_path, image_path in sorted(md_files_with_errors):
                print(f"\nFile: {rel_md_path}")
                print(f"Missing image: {image_path}")
                
except Exception as e:
    print(f"Error occurred: {str(e)}")

Processing: /Users/dereksnow/Downloads/sovai-docs/sovai-documentation/SUMMARY.md
Processing: /Users/dereksnow/Downloads/sovai-docs/sovai-documentation/README.md
Processing: /Users/dereksnow/Downloads/sovai-docs/sovai-documentation/time-series/institutional-1.md
Processing: /Users/dereksnow/Downloads/sovai-docs/sovai-documentation/time-series/time-segmentation.md
Processing: /Users/dereksnow/Downloads/sovai-docs/sovai-documentation/time-series/nowcasting-series.md
Processing: /Users/dereksnow/Downloads/sovai-docs/sovai-documentation/dashboard-examples/turing-risk-index.md
Processing: /Users/dereksnow/Downloads/sovai-docs/sovai-documentation/dashboard-examples/bankruptcy-prediction.md
Processing: /Users/dereksnow/Downloads/sovai-docs/sovai-documentation/.venv/lib/python3.12/site-packages/idna-3.10.dist-info/LICENSE.md
Processing: /Users/dereksnow/Downloads/sovai-docs/sovai-documentation/.venv/lib/python3.12/site-packages/debugpy/_vendored/pydevd/pydevd_plugins/extensions/README.md
Proces